训练前，务必选择一个fold（范围：1-4）

In [62]:
fold = 5  # 1表示在234上训练，1上验证；以此类推
if fold == 1:
    tr_set = 'tr2345'
elif fold == 2:
    tr_set = 'tr1345'
elif fold == 3:
    tr_set = 'tr1245'
elif fold == 4:
    tr_set = 'tr1235'
elif fold == 5:
    tr_set = 'tr1234'

# 设定其余参数
_boosting_type='gbdt'
_num_leaves=32
_max_depth=-1
_n_estimators=5000
_subsample=0.8
_colsample_bytree=0.8
_learning_rate=0.05
_min_child_weight=1

test_set='A'  # 1为A赛段，2为B赛段

运行以下单元格，并获取训练的predict预测文件和模型文件

In [47]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from scipy import sparse
import feather
from sklearn.feature_selection import SelectFromModel
import time
import matplotlib.pyplot as plt
from sklearn import metrics

def get_data(stage):
    print('reading data ...')
    tr1_x = feather.read_dataframe('data/tr1_x.f').drop(columns=['id','date'])
    tr2_x = feather.read_dataframe('data/tr2_x.f').drop(columns=['id','date'])
    tr3_x = feather.read_dataframe('data/tr3_x.f').drop(columns=['id','date'])
    tr4_x = feather.read_dataframe('data/tr4_x.f').drop(columns=['id','date'])
    tr5_x = feather.read_dataframe('data/tr5_x.f').drop(columns=['id','date'])
        
    tr1_y = feather.read_dataframe('data/tr1_y.f')
    tr2_y = feather.read_dataframe('data/tr2_y.f')
    tr3_y = feather.read_dataframe('data/tr3_y.f')
    tr4_y = feather.read_dataframe('data/tr4_y.f')
    tr5_y = feather.read_dataframe('data/tr5_y.f')
    
    if stage == 1:
        tr_y = pd.concat([tr2_y, tr3_y,tr4_y,tr5_y], 0)
        val_y = tr1_y
        tr_x = pd.concat([tr2_x, tr3_x,tr4_x,tr5_x], 0)
        val_x = tr1_x
    elif stage == 2:
        tr_y = pd.concat([tr1_y, tr3_y,tr4_y,tr5_y], 0)
        val_y = tr2_y
        tr_x = pd.concat([tr1_x, tr3_x,tr4_x,tr5_x], 0)
        val_x = tr2_x
    elif stage == 3:
        tr_y = pd.concat([tr1_y, tr2_y,tr4_y,tr5_y], 0)
        val_y = tr3_y
        tr_x = pd.concat([tr1_x, tr2_x,tr4_x,tr5_x], 0)
        val_x = tr3_x
    elif stage == 4:
        tr_y = pd.concat([tr1_y, tr2_y,tr3_y,tr5_y], 0)
        val_y = tr4_y
        tr_x = pd.concat([tr1_x, tr2_x,tr3_x,tr5_x], 0)
        val_x = tr4_x
    elif stage == 5:
        tr_y = pd.concat([tr1_y, tr2_y,tr3_y,tr4_y], 0)
        val_y = tr5_y
        tr_x = pd.concat([tr1_x, tr2_x,tr3_x,tr4_x], 0)
        val_x = tr5_x
    else:
        return None
    
    print('tr_x:', tr_x.shape)
    print('val_x:', val_x.shape)
    print('tr_y:', tr_y.shape)
    print('val_y:', val_y.shape)
    
    return tr_y, val_y, tr_x, val_x


In [63]:
# 读取数据
tr_y, val_y, tr_x, val_x = get_data(fold)
t1 = time.time()
# 训练
print('start time:', time.asctime(time.localtime(t1)))
clf = lgb.LGBMClassifier(
    boosting_type=_boosting_type, num_leaves=_num_leaves, reg_alpha=0.0, reg_lambda=1,
    max_depth=_max_depth, n_estimators=_n_estimators, objective='binary',
    subsample=_subsample, colsample_bytree=_colsample_bytree, subsample_freq=1,
    learning_rate=_learning_rate, min_child_weight=_min_child_weight, device='cpu', random_state=2018
)
print('fit...')
clf.fit(tr_x, tr_y, eval_set=[(val_x, val_y)], eval_metric='auc',early_stopping_rounds=50)
print('training completed!')
t2 = time.time()
print('end time:', time.asctime(time.localtime(t2)))
cost = t2-t1
tr_hour = int(cost//(60*60))
tr_miniute = int(cost%(60*60)/60)
print('time cost:{}h  {}m'.format(tr_hour, tr_miniute))

# 生成结果
last_score = float('%.6f' % clf.best_score_['valid_0']['auc'])
last_score = int(last_score*1000000)

print('saving model to data/model/{}_ml{}_mcw{}_sa{}_cs{}_{}_{}____if1_f2162_{}_{}'.format(tr_set,_num_leaves,_min_child_weight,_subsample,_colsample_bytree,_n_estimators,last_score,tr_hour, tr_miniute))
clf.booster_.save_model('data/model/{}_ml{}_mcw{}_sa{}_cs{}_{}_{}____if1_f2162_{}_{}'.format(tr_set,_num_leaves,_min_child_weight,_subsample,_colsample_bytree,_n_estimators,last_score,tr_hour, tr_miniute))




print('reading test{}...'.format(test_set))
test = feather.read_dataframe('data/test{}.f'.format(test_set))
test_x = test.drop(columns=['id', 'date'])

print('test shape:', test_x.shape)
print('predicting...')
t1 = time.time()
print('start time:', time.asctime(time.localtime(t1)))
test['score'] = clf.predict_proba(test_x)[:, 1]
del test_x
test['score'] = test['score'].apply(lambda x: float('%.6f' % x))
t2 = time.time()
print('end time:', time.asctime(time.localtime(t2)))
cost = t2-t1
hour = int(cost//(60*60))
miniute = int(cost%(60*60)/60)
print('time cost:{}h  {}m'.format(hour, miniute))
print('saving submission to data/result/test{}_predict{}_{}.csv'.format(test_set, fold, last_score))
test[['id','score']].to_csv('data/result/test{}_predict{}_{}.csv'.format(test_set, fold, last_score), index=False)
del test
print('completed!')


reading data ...
tr_x: (792000, 297)
val_x: (198006, 297)
tr_y: (792000, 1)
val_y: (198006, 1)
start time: Fri Jun 15 15:56:56 2018
fit...


C:\Users\bhneo\Anaconda3\envs\lgb\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\bhneo\Anaconda3\envs\lgb\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[1]	valid_0's auc: 0.931685
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's auc: 0.960044
[3]	valid_0's auc: 0.964509
[4]	valid_0's auc: 0.965241
[5]	valid_0's auc: 0.966504
[6]	valid_0's auc: 0.967848
[7]	valid_0's auc: 0.968161
[8]	valid_0's auc: 0.974445
[9]	valid_0's auc: 0.975455
[10]	valid_0's auc: 0.977451
[11]	valid_0's auc: 0.978679
[12]	valid_0's auc: 0.978862
[13]	valid_0's auc: 0.979236
[14]	valid_0's auc: 0.97942
[15]	valid_0's auc: 0.979557
[16]	valid_0's auc: 0.979599
[17]	valid_0's auc: 0.979702
[18]	valid_0's auc: 0.97999
[19]	valid_0's auc: 0.980292
[20]	valid_0's auc: 0.980348
[21]	valid_0's auc: 0.980577
[22]	valid_0's auc: 0.980738
[23]	valid_0's auc: 0.981139
[24]	valid_0's auc: 0.981342
[25]	valid_0's auc: 0.981365
[26]	valid_0's auc: 0.981441
[27]	valid_0's auc: 0.981661
[28]	valid_0's auc: 0.981747
[29]	valid_0's auc: 0.98197
[30]	valid_0's auc: 0.982062
[31]	valid_0's auc: 0.982122
[32]	valid_0's auc: 0.982167
[33]	valid_0's auc: 0.

[279]	valid_0's auc: 0.989632
[280]	valid_0's auc: 0.989628
[281]	valid_0's auc: 0.989634
[282]	valid_0's auc: 0.989632
[283]	valid_0's auc: 0.989641
[284]	valid_0's auc: 0.989665
[285]	valid_0's auc: 0.98967
[286]	valid_0's auc: 0.989668
[287]	valid_0's auc: 0.989672
[288]	valid_0's auc: 0.989674
[289]	valid_0's auc: 0.989658
[290]	valid_0's auc: 0.989647
[291]	valid_0's auc: 0.989653
[292]	valid_0's auc: 0.98965
[293]	valid_0's auc: 0.98966
[294]	valid_0's auc: 0.989658
[295]	valid_0's auc: 0.989682
[296]	valid_0's auc: 0.989686
[297]	valid_0's auc: 0.989691
[298]	valid_0's auc: 0.989696
[299]	valid_0's auc: 0.989713
[300]	valid_0's auc: 0.98972
[301]	valid_0's auc: 0.989742
[302]	valid_0's auc: 0.98974
[303]	valid_0's auc: 0.989744
[304]	valid_0's auc: 0.989743
[305]	valid_0's auc: 0.989736
[306]	valid_0's auc: 0.989744
[307]	valid_0's auc: 0.989755
[308]	valid_0's auc: 0.989759
[309]	valid_0's auc: 0.989784
[310]	valid_0's auc: 0.989786
[311]	valid_0's auc: 0.989794
[312]	valid_0's

[555]	valid_0's auc: 0.990608
[556]	valid_0's auc: 0.990611
[557]	valid_0's auc: 0.990612
[558]	valid_0's auc: 0.990611
[559]	valid_0's auc: 0.990619
[560]	valid_0's auc: 0.990615
[561]	valid_0's auc: 0.990612
[562]	valid_0's auc: 0.990611
[563]	valid_0's auc: 0.990614
[564]	valid_0's auc: 0.990611
[565]	valid_0's auc: 0.990633
[566]	valid_0's auc: 0.990632
[567]	valid_0's auc: 0.990642
[568]	valid_0's auc: 0.99065
[569]	valid_0's auc: 0.990649
[570]	valid_0's auc: 0.990652
[571]	valid_0's auc: 0.99065
[572]	valid_0's auc: 0.990651
[573]	valid_0's auc: 0.990651
[574]	valid_0's auc: 0.990654
[575]	valid_0's auc: 0.990651
[576]	valid_0's auc: 0.99065
[577]	valid_0's auc: 0.990656
[578]	valid_0's auc: 0.990661
[579]	valid_0's auc: 0.990664
[580]	valid_0's auc: 0.990667
[581]	valid_0's auc: 0.990672
[582]	valid_0's auc: 0.990672
[583]	valid_0's auc: 0.990675
[584]	valid_0's auc: 0.990674
[585]	valid_0's auc: 0.99067
[586]	valid_0's auc: 0.99067
[587]	valid_0's auc: 0.990674
[588]	valid_0's

[830]	valid_0's auc: 0.991119
[831]	valid_0's auc: 0.991123
[832]	valid_0's auc: 0.991127
[833]	valid_0's auc: 0.991128
[834]	valid_0's auc: 0.991129
[835]	valid_0's auc: 0.991131
[836]	valid_0's auc: 0.991131
[837]	valid_0's auc: 0.991126
[838]	valid_0's auc: 0.991127
[839]	valid_0's auc: 0.991133
[840]	valid_0's auc: 0.991136
[841]	valid_0's auc: 0.99114
[842]	valid_0's auc: 0.991139
[843]	valid_0's auc: 0.991143
[844]	valid_0's auc: 0.99114
[845]	valid_0's auc: 0.991139
[846]	valid_0's auc: 0.991135
[847]	valid_0's auc: 0.99113
[848]	valid_0's auc: 0.991131
[849]	valid_0's auc: 0.991134
[850]	valid_0's auc: 0.991136
[851]	valid_0's auc: 0.991127
[852]	valid_0's auc: 0.991125
[853]	valid_0's auc: 0.99113
[854]	valid_0's auc: 0.991125
[855]	valid_0's auc: 0.991131
[856]	valid_0's auc: 0.991133
[857]	valid_0's auc: 0.991137
[858]	valid_0's auc: 0.991137
[859]	valid_0's auc: 0.991134
[860]	valid_0's auc: 0.991142
[861]	valid_0's auc: 0.991139
[862]	valid_0's auc: 0.991136
[863]	valid_0'

[1102]	valid_0's auc: 0.991346
[1103]	valid_0's auc: 0.991343
[1104]	valid_0's auc: 0.99134
[1105]	valid_0's auc: 0.99134
[1106]	valid_0's auc: 0.991346
[1107]	valid_0's auc: 0.991342
[1108]	valid_0's auc: 0.991343
[1109]	valid_0's auc: 0.991343
[1110]	valid_0's auc: 0.991341
[1111]	valid_0's auc: 0.991342
[1112]	valid_0's auc: 0.99134
[1113]	valid_0's auc: 0.99134
[1114]	valid_0's auc: 0.99134
[1115]	valid_0's auc: 0.991341
[1116]	valid_0's auc: 0.991338
[1117]	valid_0's auc: 0.991332
[1118]	valid_0's auc: 0.991335
[1119]	valid_0's auc: 0.991335
[1120]	valid_0's auc: 0.991331
[1121]	valid_0's auc: 0.991329
[1122]	valid_0's auc: 0.991334
[1123]	valid_0's auc: 0.991337
[1124]	valid_0's auc: 0.991342
[1125]	valid_0's auc: 0.991348
[1126]	valid_0's auc: 0.991342
[1127]	valid_0's auc: 0.99134
[1128]	valid_0's auc: 0.991341
[1129]	valid_0's auc: 0.991342
[1130]	valid_0's auc: 0.991346
[1131]	valid_0's auc: 0.991351
[1132]	valid_0's auc: 0.99135
[1133]	valid_0's auc: 0.991349
[1134]	valid_0'

TPR1：当FPR等于0.001时的TPR

TPR2：当FPR等于0.005时的TPR

TPR3：当FPR等于0.01时的TPR

模型成绩 = 0.4 * TPR1 + 0.3 * TPR2 + 0.3 * TPR3

In [41]:
fpr, tpr, thresholds = metrics.roc_curve(val_y, pre, pos_label=1)

tpr1 = tpr[len(fpr[fpr <= 0.001])]
tpr2 = tpr[len(fpr[fpr <= 0.005])]
tpr3 = tpr[len(fpr[fpr <= 0.01])]

auc = 0.4*tpr1+0.3*tpr2+0.3*tpr3
auc

0.6883569979716024